In [115]:
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [116]:
diamonds=pd.read_csv('diamonds.csv')

In [117]:
cuts_lst = diamonds.cut.unique()
colors_lst = diamonds.color.unique()
clars_lst = diamonds.clarity.unique()

In [118]:
combo_lst = [[cut,color,clar] for clar in clars_lst for color in colors_lst for cut in cuts_lst]

In [119]:
dfs = [diamonds.query(f'(cut == "{x[0]}") & (color =="{x[1]}") & (clarity == "{x[2]}")') for x in combo_lst]

In [120]:
y= diamonds['price']
X= diamonds[['carat','depth','table','x','y','z']]
modelo_default = LinearRegression()
modelo_default.fit(X, y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [121]:
def ger_model(df):
    try:
        y= df['price']
        X= df[['carat','depth','table','x','y','z']]
        modelo = LinearRegression()
        modelo.fit(X, y)
        return modelo
    except:
        return modelo_default

In [122]:
modelos = [ger_model(df) for df in dfs]

In [123]:
def do_predict(row):
    cut = row['cut']
    color = row['color']
    clarity = row['clarity']
    X= [[row['carat'],row['depth'],row['table'],row['x'],row['y'],row['z']]]
    if [cut,color,clarity] in combo_lst:
        index = combo_lst.index([cut,color,clarity])
        model = modelos[index] 
        return model.predict(X)
    else:
        return modelo_default.predict(X)

In [124]:
y_predict = diamonds.apply(lambda row : do_predict(row), axis=1)

In [125]:
y_real = diamonds['price']

In [126]:
metrics.r2_score(y_real, y_predict).round(3)

0.977

In [127]:
np.sqrt(metrics.mean_squared_error(y_real,y_predict))

607.4180107363599

In [131]:
rick_diamonds = pd.read_csv('rick_diamonds.csv')
y_predict = rick_diamonds.apply(lambda row : do_predict(row)[0], axis=1)
rick_diamonds['price_predicted'] = y_predict
rick_diamonds.to_csv('./Deliverables/rick_diamonds_mean.csv')